In [1]:
# 1.23
### 将识别模块写完 初步完成
### 将API接口的多个请求都写出来
### 研究命令行参数
### 研究怎么打码

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import json
import time
import random
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from PIL import Image,ImageEnhance
import hashlib
from collections import Counter

In [4]:
def extract(html_list):
    list_cases = list(map(lambda x:x.text,html_list))
    list_cases_2 = list(map(lambda x:eval(x)["judgements"],list_cases))
    cases_3 = list(np.array(list_cases_2).flatten())
    list_case = []
    for index in range(0,10):
        for item in cases_3[index]:
            elem = item["judgement"]
            list_case.append(elem)
    return list_case

class openlaw():
    def __init__(self):
        self.AppKey = "65d54678b0ec414baf31386fa799cbca"
        self.AppSecret = "90a0af2cb7864ab28e160a89a9398436"
        self.Nonce = str("12sddrobc343fghasf5htl56")
        self.Curtime = str(int(time.time() * 1000))
    
    def openlaw_api_headers(self):
        self.Combined = self.AppSecret + self.Nonce + self.Curtime
        sha1 = hashlib.sha1(self.Combined.encode("utf-8"))
        self.CheckSum = sha1.hexdigest()
        self.headers = {"AppKey":self.AppKey,
                   "Nonce":self.Nonce,
                   "CurTime":self.Curtime,
                   "CheckSum":self.CheckSum}
        return self.headers

    def openlaw_api_search(self):
        target = "http://develop.openlaw.cn/judgement/search"
        keyword = input("请输入需要查询的案例关键词：")
        url = target + "?keyword=" + keyword
        ### 以下为选填参数
        """
        请求参数说明
        参数	参数说明
        keyword	搜索关键字
        court	法院名称
        judgeDateYear	判决年份
        """
        switch = True
        while switch == True:
            court,year = None,None
            paramater_none = input("是否需要其他设定参数（输入y/n）")
            if paramater_none == "y":
                pass
            else:
                break
            paramater = input("请输入还需要限定的参数")
            if paramater == "法院名称":
                court = input("哪个法院？")
                url = url + "&court=" + str(court)
            if paramater == "判决年份":
                year = input("哪一年?")
                url = url + "&judgeDateYear=" + str(year)
            paramater_none = input("还需要设定参数吗？（输入y/n）")
            if paramater_none == "n":
                switch = False
            else:
                switch = True
        headers = self.openlaw_api_headers()
        html = requests.get(url,headers = headers)
        return html
    
    def openlaw_api_search_v2(self,keyword):
        #### 设定法院进行爬取
        target = "http://develop.openlaw.cn/judgement/search"
        list_urls = []
        url = target + "?keyword=" + keyword
        with open("courts.txt","r",encoding = "utf-8") as f:
            court_list = eval(f.read())
        #### 选择你需要的法院
        for court in court_list[0:10]:
            url_single = url + "&court=" + str(court)
            list_urls.append(url_single)
        headers = self.openlaw_api_headers()
        html_list = list(map(lambda x:requests.get(x,headers = headers),list_urls))
        list_case = extract(html_list)
        return list_case
        
    def openlaw_api_court(self):
        target = "http://develop.openlaw.cn/judgement/court"
        dicts = {}
        keycourt = input("请输入需要查询的法院：")
        headers = self.openlaw_api_headers()
        dicts["court"] = keycourt
        html = requests.get(target,headers = headers,data = data)
        return html
        
    def openlaw_api_analysis(self):
        target = "http://develop.openlaw.cn/analytic/judgement"
        dicts = {}
        keycourt = input("请输入需要查询的关键字：")
        dicts["keyword"] = keyword
        html = requests.get(target,headers = headers,data = data)
        return html
        
o1 = openlaw()
#html_list = o1.openlaw_api_search()
#html_list = o1.openlaw_api_search_v2("合同")
pd.DataFrame(html_list).to_excel("合同100_v2.xlsx")

In [79]:
### PDF转化器 
### 核心要点抽取，各类NLP抽取（使用HanLp）
### 内容提取器(已经完成），法条统计器（已经完成）
### 判决结果相似度分析器

In [5]:
### pdf转化器
import os
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice

def pdfreader(filename):
    #获得文档对象,以二进制读方式打开
    fp = open(filename, "rb")

    #创建一个与文档关联的分析器
    parser = PDFParser(fp)

    #创建一个pdf文档的对象
    doc = PDFDocument()

    #连接解释器与文档对象
    parser.set_document(doc)
    doc.set_parser(parser)

    #初始化文档,如果文档有密码，写与此。
    doc.initialize("")

    #创建pdf资源管理器
    resource = PDFResourceManager()

    #参数分析器
    laparam = LAParams()

    #创建聚合器
    device = PDFPageAggregator(resource, laparams=laparam)

    #创建pdf页面解释器
    interpreter = PDFPageInterpreter(resource, device)

    #使用文档对象得到页面的集合
    list_text,corpus = [],[]
    for page in doc.get_pages():
        #使用页面解释器读取
        interpreter.process_page(page)
        #使用聚合器来获得内容
        layout = device.get_result()
        for out in layout:
            if hasattr(out, "get_text"):
                line = out.get_text().strip("\n ")
                line_clean = re.sub("\n","",line)
                list_text.append(line_clean)
    final_test = "".join([i for i in list_text if i != ""])
    return final_test

os.chdir("正当防卫的1000份文书")
file_list = os.listdir()
corpus = list(map(pdfreader,file_list))

In [10]:
## from PIL import Image,ImageEnhance
## 强行爬取openlaw上的法条
## 可以做一点大的改进，晚上最后来做
from bs4 import BeautifulSoup
import random

def login(dicts,login_entrance,account,password):
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(login_entrance)
    elem_account = driver.find_element_by_id("username")
    elem_account.send_keys(login_account)
    elem_password = driver.find_element_by_id("password")
    elem_password.send_keys(login_password)
    elem_verification = driver.find_element_by_id("code")
    code = input("验证码是多少:") # 手工打码
    elem_verification.send_keys(code)
    driver.find_element_by_id("submit").click()
    response = driver.page_source
    html_soup = BeautifulSoup(response,'lxml')
    return driver

def rounds(dicts,urls,login_entrance,login_account,login_password):
    driver = login(dicts,login_entrance,login_account,login_password)
    print(driver)
    list_soup = []
    index = 0 
    for url in urls[0:100]:
        driver.get(url)
        try:
            element = WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.CLASS_NAME,"entry-title"))
            )
            time.sleep(5)
        except Exception as e:
            time.sleep(5)
            pass
        html_soup = BeautifulSoup(driver.page_source,'lxml')
        list_soup.append(html_soup)
        index += 1
        if index % 10 == 0:
            time.sleep(10)
        if index % 50 == 0:
            time.sleep(50)
        if index % 100 == 0:
            pass
    return list_soup
    
dicts = {"account":["hongruzyj@hotmail.com","hongruzyj@qq.com"],"password":["945180zyj","945180zyj"]}
login_account = dicts["account"][0]
login_password = dicts["password"][0]
table = pd.read_excel("合同100.xlsx")
url_bash = "http://openlaw.cn/judgement/"
urls = list(map(lambda x:url_bash + x,table["id"]))
list_soup = rounds(dicts,urls,"http://openlaw.cn/login.jsp",login_account,login_password)

验证码是多少:fbpx2
<selenium.webdriver.chrome.webdriver.WebDriver (session="0867396bb362694f5850ceeb754100d6")>


In [16]:
table.head()

,caseNo,causes,court,courtId,docType,id,judgeDate,judgeResult,judgeWay,litigationType,sequence,title
0,（2014）华民初字第231号,NaN,华坪县人民法院,83beed44df904382a6f873f5aae50874,Verdict,c4a851324c5e4bb6b3514367a26184c8,2014-05-26,Unknow,Unknow,NaN,29728994,231商品房销售合同合同纠纷判决书
1,（2012）崇民二(商)初字第398号,['定作合同纠纷'],崇明县人民法院,45447f95b1694dc99f822187d14b97a4,Verdict,81da0b2fd5ec4d4ca22fcb1e30669858,2012-11-08,Unknow,Unknow,Civil,23879721,定作合同
2,（2009）黄民一(民)初字第3214号,NaN,上海市黄浦区人民法院,9f5a961ba6844d7593c99ce9280ca2f7,Verdict,f0101aa8e15949a081afe8ae9512b55e,2010-03-02,Unknow,Unknow,NaN,25771006,雇佣合同
3,（2016）新2123民初748号,['承揽合同纠纷'],托克逊县人民法院,d0cb859a4eda42b18c5e6bcf572e1148,Verdict,a47b7a2ac4d9419daa3e8cbf66dfd887,2016-10-31,Unknow,Unknow,Civil,14129910,段长久诉郭志刚承揽合同合同判决书
4,（2016）西法执字第00175号,['民事执行'],西平县人民法院,757834f735f24434890a9ef83da6c17a,Notification,3e8b4553ac8b4fde9a36ea52ce4bfeb2,2016-02-14,Unknow,Unknow,Execution,5336313,蔡红军执行姚云宝借款合同合同庭行通知书


In [19]:
list_corpus = []
def get_soup_text(html_soup):
    list_text_temp = []
    try:
        all_text_soup = html_soup.find_all("div",class_ = "part")
        for para in all_text_soup:
            text = re.sub("\n","",para.get_text())
            text2 = re.sub(" ","",text)
            text3 = re.sub(" ","",text2)
            list_text_temp.append(text3)
        return list_text_temp
    except Exception as e:
        return("None")

pd.DataFrame()
list_corpus = list(map(get_soup_text,list_soup))
list_corpus_text = list(map(lambda x:"".join(x),list_corpus))
list_corpus_text
table_corpus_with_id = pd.DataFrame([list_corpus_text,list(table['id'])]).T
table_corpus_with_id.columns = ["文本","文本id"]
table_final_original = table_corpus_with_id[table_corpus_with_id["文本"].str.len() > 50]
table_final_original.index = range(len(table_final_original))
table_final_original.to_excel("hetong_wenben.xlsx")

In [7]:
### 文本清理器
def clean(corpus_none_n):
    results = corpus_none_n
    ### 原告中的三类可能
    pattern_plaintiff = re.compile(r"原告：([\u4e00-\u9fa5].*?)，") # 原告第一类可能
    pattern_plaintiff_2 = re.compile(r"原告：([\u4e00-\u9fa5].*?)。") # 原告第二类可能
    pattern_plaintiff_3 = re.compile(r"原告.*?，") # 原告第三类可能

    ### 被告中的三类可能
    pattern_defender = re.compile(r"被告：.*?，")
    pattern_defender_2 = re.compile(r"被告：.*?。")

    ### 原告出生年龄
    pattern_plaintiff_birthday = re.compile(r"原告：.*\d.*?出生，")

    ### 被告出生年龄
    pattern_defender_birthday = re.compile(r"被告：.*\d.*?出生，")

    ### 立案时间
    pattern_starttime = re.compile(r"，本院于.*\d.立案后，")

    ### 诉讼请求
    pattern_claims = re.compile(r"诉讼请求：.*?。")
    pattern_claims_2 = re.compile(r"诉称，.*本院认为")

    ### 事实和理由
    pattern_facts = re.compile(r"事实和理由：.*。")

    ### 辩护事实
    pattern_defence = re.compile(r"辩称：.*")
    pattern_defence_2 = re.compile(r"辩称，.*")

    ### 法院认定事实
    pattern_court_fact_1 = re.compile(r"认定事实如下：.*。本院")
    pattern_court_fact_2 = re.compile(r"经审理查明：.*。本院")
    pattern_court_fact_3 = re.compile(r"经审理查明，.*。本院")

    ### 法院对事实的判定
    pattern_court_advise = re.compile(r"本院认为.*")

    ### 法院的意见
    pattern_court_attitude = re.compile(r"综上所述.*?。")

    ### 法条
    pattern_laws_1 = re.compile(r"《.*?》.*?条")
    pattern_laws_2 = re.compile(r"《.*?》.*?条，")

    ### 判决
    pattern_judgement = re.compile("判决如下:.*。") #需要使用去掉\n的文档

    ## 搜索原告
    def find_plaintiffs(text,pattern1,pattern2,pattern3):
        if re.search(pattern1,text):
            piece = re.search(pattern1,text).group().strip(":原告：，")
            return piece
        elif re.search(pattern2,text):
            piece = re.search(pattern2,text).group().strip(":原告：，")
            return piece
        elif re.search(pattern3,text):
            piece = re.search(pattern3,text).group().strip(":原告：，")
            return piece
        else:
            return None

    ## 搜索被告
    def find_defender(text,pattern1,pattern2):
        if re.search(pattern1,text):
            piece = re.search(pattern1,text).group().strip("被告：")
            return piece
        elif re.search(pattern2,text):
            piece = re.search(pattern2,text).group().strip("被告：")
            return piece
        else:
            return None

    ## 搜索原告出生年龄
    def find_plaintiffs_birthday(text,pattern):
        if re.search(pattern,text):
            piece = re.search(pattern,text).group().strip("，")
            text = re.sub("原告：.*，","",piece).strip("出生")
            return text
        else:
            return None

    ## 搜索被告出生年龄
    def find_defenders_birthday(text,pattern):
        if re.search(pattern,text):
            piece = re.search(pattern,text).group().strip("，")
            text = re.sub("被告：.*，","",piece).strip("出生")
            return text
        else:
            return None

    ## 立案时间
    def find_starttime(text,pattern):
        if re.search(pattern,text):
            piece = re.search(pattern,text).group().lstrip("，本院于").rstrip("立案后，")
            return piece
        else:
            return None

    ## 事实和理由
    def find_facts(text,pattern):
        if re.search(pattern,text):
            piece = re.search(pattern,text).group()
            return piece
        else:
            return None

    ## 被告辩称
    def find_defenders_words(text,pattern1,pattern2):
        if re.search(pattern1,text):
            piece = re.search(pattern1,text).group()
            return piece
        elif re.search(pattern2,text):
            piece = re.search(pattern2,text).group()
            return piece
        else:
            return None

    ## 法院认定事实
    def find_court_facts(text,pattern1,pattern2,pattern3):
        if re.search(pattern1,text):
            piece = re.search(pattern1,text).group().rstrip("本院")
            return piece
        elif re.search(pattern2,text):
            piece = re.search(pattern2,text).group().rstrip("本院")
            return piece
        elif re.search(pattern3,text):
            piece = re.search(pattern3,text).group().rstrip("本院")
            return piece
        else:
            return None

    ## 查找法条
    def find_laws(text,pattern1,pattern2):
        if re.search(pattern1,text):
            pieces = re.findall(pattern1,text)
            short_pieces_1 = list(filter(lambda x:len(x) < 50,pieces))
            pieces = re.findall(pattern2,text)
            short_pieces_2 = list(filter(lambda x:len(x) < 50,pieces))
            short_pieces = short_pieces_1 + short_pieces_2
            return short_pieces
        else:
            return None

    ## 大块文字专用
    def find_claims_or_blocks(text,pattern):
        if re.search(pattern,text):
            piece = re.search(pattern,text).group()
            return piece
        else:
            return None
    
    ## 原告诉求
    def find_claims(text,pattern1,pattern2):
        if re.search(pattern1,text):
            piece = re.search(pattern1,text).group()
            return piece
        elif re.search(pattern2,text):
            piece = re.search(pattern2,text).group()
            return piece
        
    ## 重复pattern，构造pattern list
    def repeat_pattern(pattern):
        pattern_lists = len(list(results)) * [pattern]
        return pattern_lists

    list_piece_t,list_corpus = [],[] 

    ### 是否这里可以改成(**kwargs)
    ### 原告

    plaintiffs = list(map(find_plaintiffs,corpus_none_n,repeat_pattern(pattern_plaintiff),
                          repeat_pattern(pattern_plaintiff_2),repeat_pattern(pattern_plaintiff_3)))

    ### 被告
    defenders = list(map(find_defender,corpus_none_n,repeat_pattern(pattern_defender),
                        repeat_pattern(pattern_defender_2)))
    
    ### 原告出生年龄
    plaintiffs_birthday = list(map(find_plaintiffs_birthday,corpus_none_n,repeat_pattern(pattern_plaintiff_birthday)))

    ###被告出生年龄
    defenders_birthday = list(map(find_defenders_birthday,corpus_none_n,repeat_pattern(pattern_defender_birthday)))

    ### 立案时间
    starttimes = list(map(find_starttime,corpus_none_n,repeat_pattern(pattern_starttime)))

    ### 诉讼请求
    claims = list(map(find_claims,corpus_none_n,repeat_pattern(pattern_claims),repeat_pattern(pattern_claims_2)))

    ### 事实与理由
    facts = list(map(find_claims_or_blocks,corpus_none_n,repeat_pattern(pattern_facts)))

    ### 被告辩称
    
    defences = list(map(find_defender,corpus_none_n,repeat_pattern(pattern_defence),
                        repeat_pattern(pattern_defence_2)))
    
    ### 法院认定事实
    court_facts = list(map(find_court_facts,corpus_none_n,repeat_pattern(pattern_court_fact_1),
                          repeat_pattern(pattern_court_fact_2),repeat_pattern(pattern_court_fact_3)))

    ### 法院对事实的判定
    advises = list(map(find_claims_or_blocks,corpus_none_n,repeat_pattern(pattern_court_advise)))

    ### 法院的态度
    attitudes = list(map(find_claims_or_blocks,corpus_none_n,repeat_pattern(pattern_court_attitude)))

    ### 判决
    judgement = list(map(find_claims_or_blocks,corpus_none_n,repeat_pattern(pattern_judgement)))

    ### 法条
    laws = list(map(find_laws,corpus_none_n,repeat_pattern(pattern_laws_1),
                        repeat_pattern(pattern_laws_2)))
    table_caipanwenshu = pd.DataFrame([plaintiffs,defenders,plaintiffs_birthday,defenders_birthday,
                                   starttimes,claims,facts,defences,court_facts,advises,attitudes,judgement,laws]).T
    table_caipanwenshu.columns = ["原告","被告","原告出生时间","被告出生时间","立案时间","原告诉讼请求",
                              "原告事实与理由","被告辩称","法院认定事实","法院认为","法院意见","法院判决","法条"]
    return table_caipanwenshu
### 判决后：如不服本判决，可以在判决书送达之日起十五日内，向本院递交上诉状，并按照对方当事人或者代表人的人数提出副本，上诉于北京市第一中级人民法院。
### 一审，二审要在裁判文书爬虫是就得确定好
### 案由，案件类型

In [8]:
def drop_none_columns(table):
    index_exist,index_none = 0,0
    list_keep = []
    for column in list(table.columns):
        for item in table[column]:
            if item != None:
                index_exist += 1
            else:
                index_none += 1
        ratio = float(index_none) / (float(index_exist) + 1.0) # 剃掉很多都是None的行
        if ratio<= 10:
            list_keep.append(table[column])
        else:
            pass
        index_exist = 0
        index_none = 0
    return pd.DataFrame(list_keep).T

table = clean(corpus)
table_final = table.drop(["被告","原告出生时间","被告出生时间","立案时间","原告诉讼请求","原告事实与理由"],axis = 1)
table_final
#for item in table.columns:
    #table[item]

#table_final_original["文本id"]
#table.join(table_final_original["文本id"])
#table2 = drop_none_columns(table)
#table3 = table2.join(table_final_original["文本id"])
#pd.merge(table3,table_hetong,left_on = "文本id",right_on = "id",how = "left").to_excel("final.xlsx")

,原告,被告辩称,法院认定事实,法院认为,法院意见,法院判决,法条
0,）:杨宝,辩称，对一审判决不满意，但没有上诉，不同意张金路的上诉请求。张金路从始至终没有说过一句实话，...,None,本院认为，根据《中华人民共和国侵权责任法》第六条和第十六条的规定可知，行为人因过错侵害他人民...,综上所述，张金路的上诉请求不能成立，应予驳回；一审判决认定事实清楚，适用法律正确，应予维持。,判决如下:驳回上诉，维持原判。二审案件受理费50元，由张金路负担（已交纳）。本判决为终审判决。,"[《中华人民共和国侵权责任法》第六条, 《中华人民共和国民事诉讼法》第六十四条, 《最高人民..."
1,）何某,None,None,本院认为，2003年10月17日，密云法院作出（2003）密行初字第11号行政判决，该判决在...,综上所述，何某的再审申请超过法定申请再审的期限，依照《最高人民法院关于适用&lt;中华人民共...,None,"[《最高人民法院关于适用&lt;中华人民共和国行政诉讼法&gt;的解释》第一百一十条, 《最..."
2,人李某,None,None,本院认为，被告人夏天虎无视国家法律，故意伤害他人身体健康，致一人轻伤，其行为已触犯刑律，构成...,None,判决如下:一、被告人夏天虎犯故意伤害罪，判处有期徒刑十个月，缓刑一年。第 4 页夏天虎故意伤...,"[《人损解释》第十七条, 《最高人民法院关于适用的解释》第一百三十八条, 《中华人民共和国刑..."
3,None,None,None,本院认为，上诉人张海峰与原审被告人吴过无视国家法律和社会公德，伙同他人在公共场所持械聚众斗殴...,None,None,"[《中华人民共和国刑法》第二百九十二条, 《中华人民共和国刑事诉讼法》第二百二十五条]"
4,"）:高德珍,女,1970年5月27出生汉族",None,None,本院认为，本案当事人因未能正确处理相邻关系产生矛盾，导致上诉人聂巧连与被上诉人高德珍发生吵打...,综上所述，上诉人聂巧连的上诉请求不能成立，应予驳回。,判决如下:驳回上诉，维持原判。第 3 页聂巧连、高德珍生命权、健康权、身体权纠纷二审民事判决...,"[《最高人民法院关于适用的解释》第九十条, 《中华人民共和国侵权责任法》第六条, 《中华人民..."
5,None,None,经审理查明，2015年11月3日凌晨2时许，被告人周卫祥从景谷县”浅水湾KTV”出来至门口时...,本院认为，上诉人周卫祥故意伤害他人身体致轻伤二级，其行为已构成故意伤害罪，应予以刑罚处罚。原...,None,None,"[《中华人民共和国刑法》第二百三十四条, 《中华人民共和国刑事诉讼法》第二百二十五条]"
6,人李某,辩称，其没有殴打过被害人，案发时不在现场，不应承担民事赔偿责任。 经审理查明...,经审理查明，2017年4月10日下午17时许，李某因怀疑自家畜圈的垃圾是被告人李正成家的人故...,本院认为，被告人李正成与被害人李某是同村村民，双方系邻里关系，被害人李某因怀疑李正成家人扔垃...,None,判决如下:一、被告人李正成犯故意伤害罪，判处有期徒刑一年，缓刑二年（缓刑考验期自判决执行之日...,"[《中华人民共和国刑法》第二百三十四条, 《中华人民共和国刑法》第二百三十四条, 《中华人民..."
7,人邹某1,None,None,本院认为，被告人胡某3与自诉人邹某1系翁婿关系，在平时的生产、生活中理应和睦相处，但双方因琐...,None,判决如下:一、被告人胡某3犯故意伤害罪，判处有期徒刑一年，缓刑一年零六个月。（缓刑考验期限自...,"[《中华人民共和国侵权责任法》第六条, 《最高人民法院关于适用《中华人民共和国刑事诉讼法》的..."
8,人）字某,None,None,本院认为，原审被告人李小强无视国家法律，故意伤害他人身体，其行为已经构成故意伤害罪，且致人死...,None,None,"[《中华人民共和国刑法》第二百三十四条, 《关于适用的解释》第一百五十五条, 《中华人民共和..."
9,None,None,None,None,None,None,None


In [29]:
from gensim import corpora, models, similarities
import jieba
import codecs, sys

table_judgements = table_final["法院判决"].dropna()
table_segments_list = list(map(lambda x:jieba.lcut(x),table_judgements))
stopwords = list(map(lambda x:x.strip("\n"),codecs.open('stopwords.txt', 'r', 'utf-8').readlines()))
stopwords_clean = list(map(lambda x:x.strip("\r"),stopwords))
def clean(single_para):
    list_clean = set(single_para).difference(set(stopwords_clean))
    return list_clean

list_clean = list(map(clean,table_segments_list))
texts = list_clean
# load id->word mapping (the dictionary)
dictionary = corpora.Dictionary(list_clean)
# word must appear >10 times, and no more than 40% documents
# dictionary.filter_extremes(no_below=40, no_above=0.1)
# save dictionary
dictionary.save('dict_v1.dict')

# load corpus
corpus = [dictionary.doc2bow(text) for text in texts]

# initialize a model
tfidf = models.TfidfModel(corpus)

# use the model to transform vectors, apply a transformation to a whole corpus
corpus_tfidf = tfidf[corpus]

In [45]:
import numpy as np
index = similarities.MatrixSimilarity(corpus_tfidf)
sims = index[corpus[10]]
similarity = list(sims)
np.argmax(similarity)
max_similarities = np.array(similarity).argsort()[::-1]

In [53]:
table_judgements[10]
table_judgements[607]

'判决如下:一、被告人严新忠犯故意伤害罪，判处有期徒刑十个月。（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2017年11月3日起至2018年9月2日止。         二、被告人严新忠赔偿给附带民事诉讼原告人严某1损失人民币19571.94元。（赔偿款限于本判决发生法律效力之日起三十日内支付完毕）如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向广东省肇庆市中级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份，副本二份。审判长苏永任 人民陪审员王明薇 人民陪审员黄明克 二〇一八年三月十三日 书记员郭素萍附:相关法律条文《中华人民共和国刑法》第二百三十四条故意伤害他人身体的，处三年以下有期徒刑、拘役或者管制。犯前款罪，致人重伤的，处三年以上十年以下有期徒刑；致人死亡或者以特别残忍手段致人重伤造成严重残疾的，处十年以上有期徒刑、无期徒刑或者死刑。本法另有规定的，依照规定。第六十七条犯罪以后自动投案，如实供述自己的罪行的，是自首。对于自首的犯罪分子，可以从轻或者减轻处罚。其中，犯罪较轻的，可以免除处罚。被采取强制措施的犯罪嫌疑人、被告人和正在服刑的罪犯，如实供述司法机关还未掌握的本人其他罪行的，以自首论。犯罪嫌疑人虽不具有前两款规定的自首情节，但是如实供述自己罪行的，可以从轻处罚；因其如实供述自己罪行，避免特别严重后果发生的，可以减轻处罚。《中华人民共和国刑事诉讼法》第 4 页严新忠故意伤害一审刑事判决书 - OpenLaw.CN 开放法律联盟第九十九条被害人由于被告人的犯罪行为而遭受物质损失的，在刑事诉讼过程中，有权提起附带民事诉讼。被害人死亡或者丧失行为能力的，被害人的法定代理人、近亲属有权提起附带民事诉讼。如果是国家财产、集体财产遭受损失的，人民检察院在提起公诉的时候，可以提起附带民事诉讼。最高人民法院《关于适用的解释》第一百三十八条被害人因人身权利受到犯罪侵犯或者财物被犯罪分子毁坏而遭受物质损失的，有权在刑事诉讼过程中提起附带民事诉讼；被害人死亡或者丧失行为能力的，其法定代理人、近亲属有权提起附带民事诉讼。因受到犯罪侵犯，提起附带民事诉讼或者单独提起民事诉讼要求赔偿精神损失的，人民法院不予受理。《最高人民法院关于审理人身损害赔偿案件适用法律若干问题的解释》第十九条医疗费根据医疗机构出具的医药费、住院费等收款凭证

In [96]:
### 统计法条数量
def count_laws(table):
    list_fatiao_corpus = []
    fatiao2d = list(table["法条"])
    for case in fatiao2d:
        if case != None:
            case_unrepeated = list(set(case))
            for law in case_unrepeated:
                list_fatiao_corpus.append(law)
    table_laws = pd.DataFrame([dict(Counter(list_fatiao_corpus))]).T
    table_laws.columns = ["数量"]
    table_laws_ascending = table_laws.sort_values("数量",ascending = False)
    return table_laws_ascending
count_laws(table).head()

,数量
《中华人民共和国刑法》第二百三十四条,391
《中华人民共和国刑事诉讼法》第二百二十五条,204
《中华人民共和国民事诉讼法》第二百五十三条,155
《中华人民共和国民事诉讼法》第一百七十条,130
《中华人民共和国侵权责任法》第六条,120


In [ ]:
"""
参数	参数说明
keyword	搜索关键字
court	法院名称
courtId	法院id
docType	文书类型
lawFirmId	律所id
lawFirmName	律所名称
lawyer	律师名称
lawyerId	律师id
caseNo	案号
litigant	当事人
plaintiff	原告
defendant	被告
thirdParty	第三人
judgeDateYear	判决年份
"""
"""
cookie = "s_token=1772f16933714319bc43c12f8efbfeba;"
         "c_token=f1y313371tf16933714319bc43c12f8efbfebay714319bc;"
         "Hm_lvt_a105f5952beb915ade56722dc85adf05=1548147169,1548147172,1548147188,1548147191;"
         "SESSION=OTk3ODY5OTQtZDNmOC00MDExLWIwM2YtODhkYTFmZGNhMzdj;" 
         "Hm_lpvt_a105f5952beb915ade56722dc85adf05=1548204430"
"""
"""
cookie = "s_token=901674c48bfd4c2bad98b6209f84b953; c_token=74tcw8bfdy74c48bfd4c2bad98b6209f84b9533fd4c2bad; Hm_lvt_a105f5952beb915ade56722dc85adf05=1548147169,1548147172,1548147188,1548147191; SESSION=Mzk4ZjBhYzktYjU5NC00NWY0LTkyMTItZjFhMzY4M2ZlMjhk; Hm_lpvt_a105f5952beb915ade56722dc85adf05=1548207935"
table = pd.read_excel("重婚100.xlsx")
list_url = list(map(lambda x:"http://openlaw.cn/judgement/" + x,table["id"]))
options = webdriver.ChromeOptions()
options.add_argument('Cookie="s_token=520d79ec8a224b4fa09844d4b84703e0; c_token=79ubx8a22e79ec8a224b4fa09844d4b84703e0j224b4fa0; Hm_lvt_a105f5952beb915ade56722dc85adf05=1548147169,1548147172,1548147188,1548147191; SESSION=Mzk4ZjBhYzktYjU5NC00NWY0LTkyMTItZjFhMzY4M2ZlMjhk; Hm_lpvt_a105f5952beb915ade56722dc85adf05=1548209820"')
driver = webdriver.Chrome("chromedriver.exe",chrome_options=options)
driver.get("http://openlaw.cn/judgement/1993eaec7b0543b7a2f9853f24f4b13b")
print(driver.get_cookies())
"""
"""
cookie = "Hm_lvt_a105f5952beb915ade56722dc85adf05=1548147169,1548147172,1548147188,1548147191; SESSION=MGU0Y2NhMDctYzUzMC00MTA3LWJhMjYtYjhjODBlOGVjMGU2; Hm_lpvt_a105f5952beb915ade56722dc85adf05=1548210517"
headers = {"User-Agent":ua.random,
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
           "Accept-Encoding":"gzip, deflate",
           "Accept-Language":"zh-CN,zh;q=0.9",
           "Cache-Control":"max-age=0",
           "Host":"openlaw.cn",
           "Upgrade-Insecure-Requests":"1",
           "Cookie":cookie}
"""

In [ ]:
"""
打码技术研究
import urllib.request
import re
import tesserocr
from PIL import Image
import PIL

checkcode_src = BeautifulSoup(response,'lxml').find("img",{"id":"kaptcha"})["src"]
checkcode_url = "http://openlaw.cn" + checkcode_src
urllib.request.urlretrieve(checkcode_url,"checkcode.jpg")
image = Image.open("checkcode.jpg")
image.show()
result = tesserocr.image_to_text(image)
print(result)
"""
"""
Normal_Request
def normal_request():
    list_responses = []
    for url in url_total[1:2]:
        try:
            response = requests.get(url,headers = headers)
            if response.status_code == 200:
                list_responses.append(response.text)
            else:
                continue
        except Exception as e:
            pass
"""
"""